### Passo 1: Leitura dos Dados

In [6]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

print(vinhos.head())

  classe  alcool  acido_malico  cinzas  alcalinidade_de_cinzas  magnesio  \
0      1   14.23          1.71    2.43                    15.6       127   
1      1   13.20          1.78    2.14                    11.2       100   
2      1   13.16          2.36    2.67                    18.6       101   
3      1   14.37          1.95    2.50                    16.8       113   
4      1   13.24          2.59    2.87                    21.0       118   

   fenois_totais  flavanoides  fenois_nao_flavanoides  proantocianinas  \
0           2.80         3.06                    0.28             2.29   
1           2.65         2.76                    0.26             1.28   
2           2.80         3.24                    0.30             2.81   
3           3.85         3.49                    0.24             2.18   
4           2.80         2.69                    0.39             1.82   

   intensidade_de_cor  matiz  od280_od315_de_vinhos_diluidos  prolina  
0                5.64   1.

### Passo 2: Pré-processamento dos Dados

In [7]:
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']

### Passo 3: Configuração do Experimento

In [8]:
random_states = [42, 17, 24]
ks = [3, 5]
results = []

### Passo 4: Estrutura de Cross-Validation com KFold

In [9]:
for random_state in random_states:
    print(f"\nResultados para random_state = {random_state}")
    print("=" * 50)
    
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    for k in ks:
        fold_accuracies = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            fold_accuracies.append(accuracy)

        mean_accuracy = np.mean(fold_accuracies)
        std_accuracy = np.std(fold_accuracies)
        results.append({
            'random_state': random_state,
            'k': k,
            'mean_accuracy': mean_accuracy,
            'std_accuracy': std_accuracy
        })

        print(f"k = {k}")
        print(f"  Média da Acurácia: {mean_accuracy:.2%}")
        print(f"  Desvio Padrão: {std_accuracy:.2%}")
        print("-" * 50)



Resultados para random_state = 42
k = 3
  Média da Acurácia: 70.36%
  Desvio Padrão: 13.92%
--------------------------------------------------
k = 5
  Média da Acurácia: 66.41%
  Desvio Padrão: 10.01%
--------------------------------------------------

Resultados para random_state = 17
k = 3
  Média da Acurácia: 69.87%
  Desvio Padrão: 10.40%
--------------------------------------------------
k = 5
  Média da Acurácia: 68.76%
  Desvio Padrão: 11.21%
--------------------------------------------------

Resultados para random_state = 24
k = 3
  Média da Acurácia: 71.27%
  Desvio Padrão: 9.68%
--------------------------------------------------
k = 5
  Média da Acurácia: 71.27%
  Desvio Padrão: 9.40%
--------------------------------------------------


### Passo 5: Análise dos Resultados

In [10]:
# Identificar a melhor configuração por random_state
best_results = {}

for result in results:
    rs = result['random_state']
    k = result['k']
    mean_acc = result['mean_accuracy']

    if rs not in best_results or mean_acc > best_results[rs]['mean_accuracy']:
        best_results[rs] = result
    elif mean_acc == best_results[rs]['mean_accuracy']:
        best_results[rs]['tie'] = True

# Exibir os resultados de forma organizada
print("\nResumo Final dos Resultados")
print("=" * 50)

for rs, result in best_results.items():
    print(f"Random State: {rs}")
    print(f"  Melhor k: {result['k']}")
    print(f"  Média da Acurácia: {result['mean_accuracy']:.2%}")
    print(f"  Desvio Padrão: {result['std_accuracy']:.2%}")
    if 'tie' in result:
        print("  Observação: Empate observado para esta configuração.")
    print("-" * 50)



Resumo Final dos Resultados
Random State: 42
  Melhor k: 3
  Média da Acurácia: 70.36%
  Desvio Padrão: 13.92%
--------------------------------------------------
Random State: 17
  Melhor k: 3
  Média da Acurácia: 69.87%
  Desvio Padrão: 10.40%
--------------------------------------------------
Random State: 24
  Melhor k: 3
  Média da Acurácia: 71.27%
  Desvio Padrão: 9.68%
  Observação: Empate observado para esta configuração.
--------------------------------------------------
